# Reinforced learning, työllistyminen ja Suomen sosiaaliturva

Here we implement a standardized framework for optimizing the income from work and from the Finnish Unemployment Insurance scheme. Unemployment scheme is implemented as an environment in gym_unemployment.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.5 valittaa paljon)
import warnings
warnings.filterwarnings('ignore')

pop_size=10_000

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


# Nykymalli 

Lasketaan työllisyysasteet nykymallissa.

In [3]:
cc1=Lifecycle(EK=False,minimal=False,mortality=False,perustulo=False)
cc1.run_RL(train=True,steps=2_000_000,cont=False,rlmodel='acktr',save='perusmalli_4',\
           batch=1,pop=pop_size)

(26,) (26,)
No mortality included




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






training...








---------------------------------
| explained_variance | 0.00149  |
| fps                | 1163     |
| nupdates           | 1        |
| policy_entropy     | 1.39     |
| policy_loss        | 59.5     |
| total_timesteps    | 0        |
| value_loss         | 1.88e+03 |
---------------------------------
---------------------------------
| explained_variance | -0.313   |
| fps                | 1778     |
| nupdates           | 100      |
| policy_entropy     | 1.32     |
| policy_loss        | 0.688    |
| total_timesteps    | 243639   |
| value_loss         | 6.43     |
---------------------------------
---------------------------------
| explained_variance | 0.953    |
| fps                | 

KeyboardInterrupt: 

In [ ]:
#cc1=Lifecycle(EK=False,minimal=False,mortality=False,perustulo=False)
#cc1.run_RL(train=True,steps=20_000_000,cont=False,rlmodel='acer',save='perusmalli_4',\
#           batch=1,pop=pop_size)

# EK:n malli

Lasketaan vertailukelpoiset työllisyysasteet EK:n ehdottamalla mallilla. Mallissa on toteuttu muutoksia ansiosidonnaiseen työttömyysturvaan, asumistukeen, toimeentulotukeen ja verotukseen.

In [ ]:
cc2=Lifecycle(EK=True,minimal=False)
cc2.run_RL(train=True,steps=25_000_000,cont=False,rlmodel='lstm',save='ekn_malli',pop=pop_size,start_from='perusmalli_4')

In [ ]:
cc2.compare_with(cc1)

# 300 vs 400 päivän kesto ansiosidonnaisessa

Mikä on alle kolmen vuoden työhistorian lyhyemmän (300 pv) ansiosidonnaisen päivärahan enimmäiskeston vaikutus työllisyyteen? Kokeillaan miten työllisyyteen vaikuttaisi sen pidentäminen 400 päivään.

In [ ]:
#cc1_400=Lifecycle(EK=False,minimal=False,ansiopvraha_kesto300=400)
#cc1_400.run_RL(train=True,steps=25_000_000,cont=False,rlmodel='acktr',save='300_vs_400',pop=pop_size,start_from='perusmalli')

In [ ]:
#cc1_400.compare_with(cc1)

# Työssäoloehdon pituus 12 kk

Entä jos työssäoloehto olisikin 12 kuukautta pitkä nykyisen 6 kuukauden sijaan?

In [ ]:
cc1_toe=Lifecycle(EK=False,minimal=False,ansiopvraha_toe=1.0)
cc1_toe.run_RL(train=True,steps=25_000_000,cont=False,rlmodel='lastm',save='toe_12',pop=pop_size,start_from='perusmalli_4')

In [ ]:
cc1_toe.compare_with(cc1)

# Lomarahojen jaksotus

In [ ]:
#TBD

# Ansiosidonnaisen päivärahan lyhennys 50 pv

Tarkastellaan, miten työllisyyteen vaikuttaisi ansiosidonnaisen päivärahan lyhentäminen 50 päivällä. Tällöin alle kolmen vuoden työhistorialla ansiosidonnaisen päivärahan enimmäiskesto olisi 250 pv ja pidemmällä työhistorialla enimmäiskesto olisi 350 pv.

In [ ]:
cc1_350=Lifecycle(EK=False,minimal=False,ansiopvraha_kesto300=250,ansiopvraha_kesto400=350)
cc1_350.run_RL(train=True,steps=25_000_000,cont=False,rlmodel='lastm',save='50_short',pop=pop_size,start_from='perusmalli_4')

In [ ]:
cc1_350.compare_with(cc1)

# Perustulo

In [ ]:
cc2=Lifecycle(EK=False,minimal=False)
cc2.run_RL(train=True,steps=10_000_000,cont=False,rlmodel='lastm',save='perustulo',\
           batch=1,pop=pop_size,perustulo=True)

In [ ]:
cc2.compare_with(cc1)